In [1]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.2-BETA4 (09/Jan/2022)


# PartialFullfilment (NBTest 48)

NBTEST: NOTEST_DEFAULT = TEST

## PartialFullfilment

Initialize a fast simulator

In [2]:
Sim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, raiseonerror=True)

In [3]:
Sim.add_strategy("ETH", 10, 2000, 2000, 10000, 1000, 1000)
Sim.add_strategy("ETH", 10, 2010, 2010, 10000, 1010, 1010)
Sim.add_strategy("ETH", 10, 2020, 2020, 10000, 1020, 1020)
Sim.add_strategy("ETH", 10, 2030, 2030, 10000, 1030, 1030)
Sim.add_strategy("ETH", 10, 2040, 2040, 10000, 1040, 1040)
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,10.0,10.0,ETH,False,2000.0,2000.0,2000.0,USDC per ETH,1
1,1,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1000.0,1000.0,1000.0,USDC per ETH,0
2,2,ETHUSDC,ETH,10.0,10.0,ETH,False,2010.0,2010.0,2010.0,USDC per ETH,3
3,3,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1010.0,1010.0,1010.0,USDC per ETH,2
4,4,ETHUSDC,ETH,10.0,10.0,ETH,False,2020.0,2020.0,2020.0,USDC per ETH,5
5,5,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1020.0,1020.0,1020.0,USDC per ETH,4
6,6,ETHUSDC,ETH,10.0,10.0,ETH,False,2030.0,2030.0,2030.0,USDC per ETH,7
7,7,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1030.0,1030.0,1030.0,USDC per ETH,6
8,8,ETHUSDC,ETH,10.0,10.0,ETH,False,2040.0,2040.0,2040.0,USDC per ETH,9
9,9,ETHUSDC,USDC,10000.0,10000.0,USDC,False,1040.0,1040.0,1040.0,USDC per ETH,8


### Route by Source

#### AMM buys ETH

Start with an acceptable amount and support_partial set to False

In [4]:
results = Sim.amm_buys('ETH',10, execute=False, support_partial=False)['trades']  # route_trade_by_source
results

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #9,False,False,False,None,10000.000000,USDC,9.615385,ETH,ETHUSDC,9,1,1039.615385,USDC per ETH
0,0.1,0,1,route #7,False,False,False,None,396.153846,USDC,0.384615,ETH,ETHUSDC,7,1,1039.615385,USDC per ETH
0,0,0,A,AMM sells 10396USDC buys 10ETH,True,False,False,None,10396.153846,USDC,10.000000,ETH,ETHUSDC,"[9, 7]",2,1039.615385,USDC per ETH


Then try an unacceptable amount with support_partial still set to False

Should FAIL

In [5]:
try:
    results = Sim.amm_buys('ETH',1000000, execute=False, support_partial=False)['trades']  # route_trade_by_source
except AssertionError as e:
    print('Should FAIL with insufficient liquidity...')
    assert(True)
    print(e)

Should FAIL with insufficient liquidity...
Insufficient liquidity across all user positions to support this trade.


Then set support_partial = True and pass an acceptable amount

In [6]:
results = Sim.amm_buys('ETH',10, execute=False, support_partial=True)['trades']  # route_trade_by_source
results

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1.0,1,0,route #9,False,False,False,None,10000.000000,USDC,9.615385,ETH,ETHUSDC,9,1,1039.615385,USDC per ETH
0,1.1,1,1,route #7,False,False,False,None,396.153846,USDC,0.384615,ETH,ETHUSDC,7,1,1039.615385,USDC per ETH
0,1,1,A,AMM sells 10396USDC buys 10ETH,True,False,False,None,10396.153846,USDC,10.000000,ETH,ETHUSDC,"[9, 7]",2,1039.615385,USDC per ETH


Then set support_partial = True and pass an unacceptable amount

Should NOT FAIL

In [7]:
results = Sim.amm_buys('ETH',10000000, execute=False, support_partial=True)['trades']  # route_trade_by_source
assert(f"{results[results.uid=='2']['amt1'].values[0]:.0f}" == "50000")
results

Assuming partial fulfillment.


,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2.0,2,0,route #9,False,False,False,None,10000.0,USDC,9.615385,ETH,ETHUSDC,9,1,1019.803895,USDC per ETH
0,2.1,2,1,route #7,False,False,False,None,10000.0,USDC,9.708738,ETH,ETHUSDC,7,1,1019.803895,USDC per ETH
0,2.2,2,2,route #5,False,False,False,None,10000.0,USDC,9.803922,ETH,ETHUSDC,5,1,1019.803895,USDC per ETH
0,2.3,2,3,route #3,False,False,False,None,10000.0,USDC,9.900990,ETH,ETHUSDC,3,1,1019.803895,USDC per ETH
0,2.4,2,4,route #1,False,False,False,None,10000.0,USDC,10.000000,ETH,ETHUSDC,1,1,1019.803895,USDC per ETH
0,2,2,A,AMM sells 50000USDC buys 49ETH,True,False,True,None,50000.0,USDC,49.029034,ETH,ETHUSDC,"[9, 7, 5, 3, 1]",5,1019.803895,USDC per ETH


Now test the other combinations

#### AMM buys USDC

In [8]:
try:
    results = Sim.amm_buys('USDC',1000000, execute=False, support_partial=False)['trades']  # route_trade_by_source
except AssertionError as e:
    print('Should FAIL with insufficient liquidity...')
    assert(True)
    print(e)

Should FAIL with insufficient liquidity...
Insufficient liquidity across all user positions to support this trade.


In [9]:
results = Sim.amm_buys('USDC',10000000, execute=False, support_partial=True)['trades']  # route_trade_by_source
assert(f"{results[results.uid=='3']['amt1'].values[0]:.0f}" == "50")
results

Assuming partial fulfillment.


,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3.0,3,0,route #0,False,False,False,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2020.0,USDC per ETH
0,3.1,3,1,route #2,False,False,False,None,10.0,ETH,20100.0,USDC,ETHUSDC,2,1,2020.0,USDC per ETH
0,3.2,3,2,route #4,False,False,False,None,10.0,ETH,20200.0,USDC,ETHUSDC,4,1,2020.0,USDC per ETH
0,3.3,3,3,route #6,False,False,False,None,10.0,ETH,20300.0,USDC,ETHUSDC,6,1,2020.0,USDC per ETH
0,3.4,3,4,route #8,False,False,False,None,10.0,ETH,20400.0,USDC,ETHUSDC,8,1,2020.0,USDC per ETH
0,3,3,A,AMM sells 50ETH buys 101000USDC,True,False,True,None,50.0,ETH,101000.0,USDC,ETHUSDC,"[0, 2, 4, 6, 8]",5,2020.0,USDC per ETH


### Route by Target

#### AMM sells ETH

In [10]:
try:
    results = Sim.amm_sells('ETH',1000000, execute=False, support_partial=False)['trades']  # route_trade_by_source
except AssertionError as e:
    print('Should FAIL with insufficient liquidity...')
    assert(True)
    print(e)

Should FAIL with insufficient liquidity...
Insufficient liquidity across all user positions to support this trade.


In [11]:
results = Sim.amm_sells('ETH',10000000, execute=False, support_partial=True)['trades']  # route_trade_by_source
assert(f"{results[results.uid=='4']['amt1'].values[0]:.0f}" == "50")
results

Assuming partial fulfillment.
Setting trade amount to max: 50


,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,4.0,4,0,route #0,False,False,False,None,10.0,ETH,20000.0,USDC,ETHUSDC,0,1,2020.0,USDC per ETH
0,4.1,4,1,route #2,False,False,False,None,10.0,ETH,20100.0,USDC,ETHUSDC,2,1,2020.0,USDC per ETH
0,4.2,4,2,route #4,False,False,False,None,10.0,ETH,20200.0,USDC,ETHUSDC,4,1,2020.0,USDC per ETH
0,4.3,4,3,route #6,False,False,False,None,10.0,ETH,20300.0,USDC,ETHUSDC,6,1,2020.0,USDC per ETH
0,4.4,4,4,route #8,False,False,False,None,10.0,ETH,20400.0,USDC,ETHUSDC,8,1,2020.0,USDC per ETH
0,4,4,A,AMM sells 50ETH buys 101000USDC,True,False,True,None,50.0,ETH,101000.0,USDC,ETHUSDC,"[0, 2, 4, 6, 8]",5,2020.0,USDC per ETH


#### AMM sells USDC

In [12]:
try:
    results = Sim.amm_sells('USDC',1000000, execute=False, support_partial=False)['trades']  # route_trade_by_source
except AssertionError as e:
    print('Should FAIL with insufficient liquidity...')
    assert(True)
    print(e)

Should FAIL with insufficient liquidity...
Insufficient liquidity across all user positions to support this trade.


In [13]:
results = Sim.amm_sells('USDC',10000000, execute=False, support_partial=True)['trades']  # route_trade_by_source
assert(f"{results[results.uid=='5']['amt1'].values[0]:.0f}" == "50000")
results

Assuming partial fulfillment.
Setting trade amount to max: 50000


,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,5.0,5,0,route #9,False,False,False,None,10000.0,USDC,9.615385,ETH,ETHUSDC,9,1,1019.803895,USDC per ETH
0,5.1,5,1,route #7,False,False,False,None,10000.0,USDC,9.708738,ETH,ETHUSDC,7,1,1019.803895,USDC per ETH
0,5.2,5,2,route #5,False,False,False,None,10000.0,USDC,9.803922,ETH,ETHUSDC,5,1,1019.803895,USDC per ETH
0,5.3,5,3,route #3,False,False,False,None,10000.0,USDC,9.900990,ETH,ETHUSDC,3,1,1019.803895,USDC per ETH
0,5.4,5,4,route #1,False,False,False,None,10000.0,USDC,10.000000,ETH,ETHUSDC,1,1,1019.803895,USDC per ETH
0,5,5,A,AMM sells 50000USDC buys 49ETH,True,False,True,None,50000.0,USDC,49.029034,ETH,ETHUSDC,"[9, 7, 5, 3, 1]",5,1019.803895,USDC per ETH
